#Introduction

This notebook will help you to train yolov3 and yolov4, also their tiny versions with applying different hyperparameters and create a different instances for each project training.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1brPZDy_yVDo38ixxpI6iSu7V4fj82Ohq?usp=sharing)

In [ ]:
import os
from shutil import copyfile

###Link Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Move to the path where you want to save in gdrive

In [ ]:
%cd /content/drive/MyDrive  

###Clone the repo   ==> <font color='red'>RunFirstTime</font>  when starting from zero

In [ ]:
!git clone https://github.com/MNaseerSubhani/TheEasyYolo.git

### Get inside the repo folder

In [ ]:
%cd TheEasyYolo

###Update Submodule : <font color='red'>RunFirstTime</font>  when starting from zero

In [ ]:
!git submodule init
!git submodule update

###  <font color='yellow'>Yolo Instance Init</font> : <font color='red'>RunFirstTime</font>  when creating new Instance

In [ ]:
%cd /content/drive/MyDrive/TheEasyYolo

In [ ]:
instance_name = "yolo_stage_2"     # Put any name of the Instance 
#set hyperparameters
num_of_classes = 1                  # Total number of classes 
channel = 1                         # Channel used "1 for Grayscale or 3 for RGB"
sub_division = 16            
width = 320                         # Width of input image
height = 256                        # Height of input image
batch = 16                          # batch size
yolo_ver = "YoloV4-tiny"            #The input should be YoloV4, YoloV4-tiny, YoloV3, YoloV3-tiny


In [ ]:
if not os.path.exists(instance_name):
  os.mkdir(instance_name)

In [ ]:
if yolo_ver == "YoloV3-tiny":
  cfg_name = "yolov3-tiny.cfg"
elif yolo_ver == "YoloV3":
  cfg_name = "yolov3.cfg"
elif yolo_ver == "YoloV4-tiny":
  cfg_name = "yolov4-tiny.cfg"
elif yolo_ver == "YoloV4":
  cfg_name = "yolov4.cfg"

In [ ]:
cfg_file_name = f'{cfg_name[:-4]}_{instance_name}.cfg'
copyfile(os.path.join('./darknet/cfg/', cfg_name),os.path.join(instance_name, cfg_file_name))

In [ ]:
%cd {instance_name}
!sed -i 's/subdivisions=1/subdivisions=$sub_division/' {cfg_file_name}
!sed -i 's/width=416/width=$width/' {cfg_file_name}
!sed -i 's/batch=64/batch=$batch/' {cfg_file_name}
!sed -i 's/height=416/height=$height/' {cfg_file_name}
!sed -i 's/channels=3/channels=$channel/' {cfg_file_name}
!sed -i 's/classes=80/classes=$num_of_classes/' {cfg_file_name}
filters = int ((num_of_classes + 5) * 3)
!sed -i 's/filters=255/filters=$filters/' {cfg_file_name}
%cd ..

In [ ]:
copyfile('generate_test.py',os.path.join(instance_name, 'generate_test.py'))
copyfile('generate_train.py',os.path.join(instance_name, 'generate_train.py'))

###  <font color='red'>NOTE: At this stage add dataset to the instance folder as structured below</font>

--data

  ----------- train

  ----------- test

### Generate text files for the data : <font color='red'>RunFirstTime</font>  when creating new Instance

In [ ]:
%cd {instance_name}
!python generate_train.py
!python generate_test.py
%cd ..

###Generate train.names file for the instance : <font color='red'>RunFirstTime</font>  when creating new Instance

In [ ]:
%cd {instance_name}
with open("train.names", "w") as f:   
    f.write("defect")
    #f.write("\nother classe")
%cd ..

###Generate train.data file for the instance : <font color='red'>RunFirstTime</font>  when creating new Instance

In [ ]:
%cd {instance_name}

if not os.path.exists('backup'):
  os.mkdir('backup')

with open("train.data", "w") as f:   
    f.write(f"classes={num_of_classes}")
    f.write(f"\ntrain = {os.path.join(os.getcwd(), 'data', 'train.txt')}")
    f.write(f"\nvalid = {os.path.join(os.getcwd(), 'data', 'test.txt')}")
    f.write(f"\nnames = {os.path.join(os.getcwd(), 'train.names')}")
    f.write(f"\nbackup = {os.path.join(os.getcwd(), 'backup/')}")
    #f.write("\nother classe")
%cd ..

#Training

In [ ]:
%cd darknet

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

### Download pre-trained weights for the convolutional layers : <font color='red'>RunFirstTime</font>  when creating new Instance

In [ ]:
if yolo_ver == "YoloV4" or yolo_ver == "YoloV4-tiny":
  !wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137
  pre_trained_weights = 'yolov4.conv.137'
elif yolo_ver == "YoloV3":
  !wget https://github.com/smarthomefans/darknet-test/blob/master/yolov3-tiny.weights
  pre_trained_weights = 'yolov3-tiny.weights'
elif yolo_ver == "YoloV3-tiny":
  !wget https://github.com/smarthomefans/darknet-test/blob/master/yolov3-tiny.weights
  pre_trained_weights = 'yolov3-tiny.weights'

### <font color='yellow'>Run Make</font> : <font color='red'>RunFirstTime</font>  when starting from zero

In [ ]:
!make

###Training 

In [ ]:
!./darknet detector train ../{instance_name}/train.data ../{instance_name}/{cfg_file_name} {pre_trained_weights} -dont_show -map -clear

Continue Training

In [ ]:
!./darknet detector train ../{instance_name}/train.data ../{instance_name}/{cfg_file_name} ../{instance_name}/backup/{cfg_file_name[:-4] + '_last.weights'} -dont_show -map -clear

#Test

In [ ]:
%cd ..

In [ ]:
%cd {instance_name}
!sed -i 's/batch=$batch/batch=1/' {cfg_file_name}
!sed -i 's/subdivisions=$sub_division/subdivisions=1/' {cfg_file_name}
%cd ..

In [ ]:
%cd darknet

In [ ]:
!./darknet detector test ../{instance_name}/train.data ../{instance_name}/{cfg_file_name} ../{instance_name}/backup/{cfg_file_name[:-4] + '_last.weights'} ../{instance_name}/data/test/IMG_0296.jpg -thresh 0.1

In [ ]:
%cd home/TheEasyYolo

In [ ]:
!jupyter nbconvert --ClAearOutputPreprocessor.enabled=True --inplace TheEasyYolo.ipynb

In [ ]:
!ls